 Name: Aditi Hoskere Deepak
 
 USC ID: 6076494222
 
                                        INF 552 HW7
                                        
                                         Problem 1

In [1]:
import os
import random
import sys

import cv2
import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sp
import seaborn as sns
import tensorflow as tf
import tqdm
from keras import regularizers
from keras.callbacks import ModelCheckpoint
from keras.datasets import cifar10
from keras.layers import (
    Activation,
    Conv2D,
    Dense,
    Dropout,
    Flatten,
    MaxPooling2D,
    Reshape,
    Softmax,
)
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from scipy.spatial.distance import euclidean
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.model_selection import train_test_split
from sklearn.neighbors import DistanceMetric
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tensorflow.python.client import device_lib



Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
config.gpu_options.allow_growth = True
config.gpu_options.visible_device_list = "0" #only the gpu 0 is allowed
tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8413874220412450758
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 1063625630893394599
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 846040070640666993
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5121520435
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16653474451379792256
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
]


i). Concatenate your text files to create a corpus of Russell's writings.

In [3]:
import os
path="../Data"
final_file=open("../Data/new_output.txt",'w')
directory_list=os.listdir(path)
for dir1 in directory_list:
    filepath=path+"/"+dir1
    with open(filepath,'r',encoding='ascii',errors='ignore') as new: 
        for line in new:
            final_file.write(line)
final_file.close()

ii). Use a character-level representation for this model by using extended ASCII that has N = 256 characters. Each character will be encoded into a an integer using its ASCII code. Rescale the integers to the range [0; 1]

In [4]:
corpus_path="../Data/new_output.txt"
from sklearn.preprocessing import MinMaxScaler
import string
dictionary={}
value_list=[]
scaled_dict={}
char_dict={}
char_list=[]
a=set()
count1=0
punctuation=["|","'","/","[","+","(",".","&","=",'!',"?","{","<","-","*","%","}","_","^",",","#","~",";","$",">",":",")","]",'"','\\']
with open(corpus_path,'r') as f:
    f=list(f)
    for i in f:
        for j in i:
            j=j.lower()
            if j in punctuation:
                continue
            else:
                char_list.append(j)
            a.add(j)
            for l in a:
                dictionary[l]=count1
                count1+=1

count=0
for ke,va in dictionary.items():
    char_dict[count]=ke
    count+=1

print("The values before scaling - ")
print(dictionary)

print("The values after scaling - ")
scaler=MinMaxScaler()
for key,value in dictionary.items():
    value_list.append(value)
scaler.fit(np.array(value_list).reshape(-1,1))


for key,val in dictionary.items():
    reshaped_array=np.array(val).reshape(1,-1)
    val=scaler.transform(reshaped_array)
    scaled_dict[key]=val[0][0]
print(scaled_dict)

print("BAck to character - ")
print(char_dict)

The values before scaling - 
{'3': 0, 'm': 1, 'd': 2, 'n': 3, '8': 4, '5': 5, 'g': 6, 'o': 7, 'u': 8, 't': 9, 'q': 10, '1': 11, 'a': 12, 'l': 13, 's': 14, 'e': 15, 'h': 16, '2': 17, '\n': 18, 'f': 19, '0': 20, 'j': 21, '6': 22, 'w': 23, 'k': 24, 'y': 25, 'v': 26, 'b': 27, 'x': 28, 'i': 29, '9': 30, 'c': 31, '7': 32, 'r': 33, 'z': 34, 'p': 35, '4': 36, ' ': 37}
The values after scaling - 
{'3': 0.0, 'm': 0.02702702702702703, 'd': 0.05405405405405406, 'n': 0.08108108108108109, '8': 0.10810810810810811, '5': 0.13513513513513514, 'g': 0.16216216216216217, 'o': 0.1891891891891892, 'u': 0.21621621621621623, 't': 0.24324324324324326, 'q': 0.2702702702702703, '1': 0.2972972972972973, 'a': 0.32432432432432434, 'l': 0.35135135135135137, 's': 0.3783783783783784, 'e': 0.40540540540540543, 'h': 0.43243243243243246, '2': 0.4594594594594595, '\n': 0.4864864864864865, 'f': 0.5135135135135136, '0': 0.5405405405405406, 'j': 0.5675675675675675, '6': 0.5945945945945946, 'w': 0.6216216216216217, 'k': 0.648

iii). Choose a window size, e.g., W = 100.

iv). Inputs to the network will be the first 99 characters of each sequence, and the output of the network will be the Wth character of the sequence.

In [5]:
#Choosing a window size of 100 and taking the first 99 as input and the 100th character as output
#This is repeated for the entire length of the dataset

size=len(char_list)-100
window=100
input_data=[]
output_data=[]

for i in range(0,size,1):
    inputdata=char_list[i:window-1]
    outputdata=char_list[window]
    input_data.append([scaled_dict[j] for j in inputdata])
    output_data.append(dictionary[outputdata])
    window=window+1

1c).

v),vi),vii),viii)

In [6]:
print(len(input_data))
print(len(output_data))
output_data=keras.utils.to_categorical(output_data)
output_data.shape

3065304
3065304


(3065304, 38)

In [7]:
from keras.layers import LSTM
num_classes=38
data=np.reshape(input_data,(len(input_data),99,1))
model=Sequential()
model.add(LSTM(units=256,input_shape=(data.shape[1], data.shape[2])))
model.add(Dense(num_classes,activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 256)               264192    
_________________________________________________________________
dense_1 (Dense)              (None, 38)                9766      
Total params: 273,958
Trainable params: 273,958
Non-trainable params: 0
_________________________________________________________________


In [8]:
from keras.optimizers import Adam
learning_rate=0.0001
opt=Adam(lr=learning_rate)
model.compile(loss="categorical_crossentropy",optimizer=opt,metrics=["accuracy"])

x. Use model checkpointing to keep the network weights to determine each time an improvement in loss is observed at the end of the epoch. Find the best set of weights in terms of loss.

In [10]:
filepath_checkpoint='../Data/new_model.hdf5'
checkpoint_callback=ModelCheckpoint(filepath=filepath_checkpoint,save_weights_only=True,monitor='loss',mode='min')
model.fit(data,output_data,epochs=6,batch_size=512,callbacks=[checkpoint_callback])

Epoch 1/6
3065304/3065304 [==============================] - 920s 300us/step - loss: 2.8604 - accuracy: 0.1702
Epoch 2/6
3065304/3065304 [==============================] - 925s 302us/step - loss: 2.8326 - accuracy: 0.1751
Epoch 3/6
3065304/3065304 [==============================] - 919s 300us/step - loss: 2.8263 - accuracy: 0.1758
Epoch 4/6
3065304/3065304 [==============================] - 941s 307us/step - loss: 2.8098 - accuracy: 0.1809
Epoch 5/6
3065304/3065304 [==============================] - 936s 305us/step - loss: 2.7974 - accuracy: 0.1849
Epoch 6/6
3065304/3065304 [==============================] - 939s 306us/step - loss: 2.7814 - accuracy: 0.1876


xi). Use the network with the best weights to generate 1000 characters,using the following text as initialization of the network:

In [11]:
filepath='../Data/new_model.hdf5'
model.load_weights(filepath)
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001))
model.summary()
prediction_data='There are those who take mental phenomena naively, just as they would physical phenomena. This school of psychologists tends not to emphasize the object.'

punctuation=["|","'","/","[","+","(",".","&","=",'!',"?","{","<","-","*","%","}","_","^",",","#","~",";","$",">",":",")","]",'"','\\']
data_list=[]
for i in prediction_data:
    for j in i:
        j=j.lower()
        if j in punctuation:
            continue
        else:
            data_list.append(j)


new_list=[]
for char in data_list[-99:]:
    new_list.append(scaled_dict[char])

generated_text=''
for s in range(1,1001,1):
    length_pred=len(new_list)
    new_data=np.reshape(new_list,(1,length_pred,1))
    pred=model.predict(new_data)
    val=np.argmax(pred)
    character=char_dict[val]
    generated_text += character
    new_list.append(scaled_dict[character])
    new_list=new_list[1:len(new_list)]

print("THe generated data is - ")  
print(generated_text)


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 256)               264192    
_________________________________________________________________
dense_1 (Dense)              (None, 38)                9766      
Total params: 273,958
Trainable params: 273,958
Non-trainable params: 0
_________________________________________________________________
THe generated data is - 
 n e hee n e ae   aeeie e e
t het hee ne he   aeeie e e h e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e

In [12]:
prediction_data+generated_text

'There are those who take mental phenomena naively, just as they would physical phenomena. This school of psychologists tends not to emphasize the object. n e hee n e ae   aeeie e e\nt het hee ne he   aeeie e e h e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e e